# 電話番号フィルタリング問題の解説

この問題を段階的に解説し、bash one-linerで解決します。

## 問題の理解

**有効な電話番号の形式:**
1. `(xxx) xxx-xxxx` - カッコ付き形式
2. `xxx-xxx-xxxx` - ハイフン形式

ここで `x` は数字(0-9)を表します。

## 解決策

In [ ]:
#!/bin/bash

# Solution 1: Using grep with extended regex
# Analyze Complexity
# Runtime 63 ms
# Beats 58.79%
# Memory 3.47 MB
# Beats 84.10%

grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt

# Solution 2: Using sed (alternative)
# Analyze Complexity
# Runtime 65 ms
# Beats 45.95%
# Memory 3.42 MB
# Beats 84.10%

sed -n -E '/^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$/p' file.txt

# Solution 3: Using awk (alternative)
# Analyze Complexity
# Runtime 71 ms
# Beats 15.78%
# Memory 3.84 MB
# Beats 3.13%

awk '/^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$/' file.txt

## 最もシンプルな解答

```bash
grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt
```

---

## 詳細な図解による解説

### 1. 正規表現パターンの構造

```
^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$
```

この正規表現を分解して理解しましょう:

#### **全体構造**
```
    ^                                                              $
    |                                                              |
  行頭                        OR演算子                            行末
         ┌──────────────────┴──────────────────┐
         |                                      |
   パターン1                                パターン2
  xxx-xxx-xxxx                          (xxx) xxx-xxxx
```

#### **パターン1: `[0-9]{3}-[0-9]{3}-[0-9]{4}`**

```
  [0-9]{3}    -    [0-9]{3}    -    [0-9]{4}
     │        │       │        │       │
  3桁の数字  ハイフン 3桁の数字  ハイフン 4桁の数字
     │                │              │
    987              123            4567

例: 987-123-4567
```

#### **パターン2: `\([0-9]{3}\) [0-9]{3}-[0-9]{4}`**

```
  \(  [0-9]{3}  \)  スペース  [0-9]{3}  -  [0-9]{4}
   │     │      │      │        │      │     │
  左カッコ 3桁 右カッコ 空白    3桁  ハイフン 4桁
   │     │      │              │            │
   (    123     )             456          7890

例: (123) 456-7890
```

**重要ポイント:** `\(` と `\)` はエスケープが必要です(カッコ自体を表すため)

---

### 2. grep コマンドの動作フロー

```
┌─────────────────────────────────────────────────┐
│              file.txt の内容                     │
├─────────────────────────────────────────────────┤
│  行1: 987-123-4567                              │
│  行2: 123 456 7890                              │
│  行3: (123) 456-7890                            │
├─────────────────────────────────────────────────┤
              ↓ grep -E で各行をチェック
├─────────────────────────────────────────────────┤
│              パターンマッチング                   │
├─────────────────────────────────────────────────┤
│  行1: 987-123-4567                              │
│       ✓ パターン1にマッチ → 出力                 │
│                                                 │
│  行2: 123 456 7890                              │
│       ✗ どちらのパターンにもマッチしない          │
│       (スペースがハイフンではない)                │
│                                                 │
│  行3: (123) 456-7890                            │
│       ✓ パターン2にマッチ → 出力                 │
├─────────────────────────────────────────────────┤
              ↓ 結果出力
├─────────────────────────────────────────────────┤
│  987-123-4567                                   │
│  (123) 456-7890                                 │
└─────────────────────────────────────────────────┘
```

---

### 3. オプションの説明

```bash
grep -E '^pattern$' file.txt
     │   │         └─ 入力ファイル
     │   └─ 正規表現パターン
     └─ Extended Regular Expression (拡張正規表現)
```

- **`-E`**: 拡張正規表現を使用(+, ?, |, () などが使える)
- **`^`**: 行頭にマッチ(余分な文字がないことを保証)
- **`$`**: 行末にマッチ(余分な文字がないことを保証)

---

### 4. テストケースの検証

#### **有効な番号**
```
✓ 987-123-4567
  [0-9]{3}-[0-9]{3}-[0-9]{4} にマッチ
  
✓ (123) 456-7890
  \([0-9]{3}\) [0-9]{3}-[0-9]{4} にマッチ
```

#### **無効な番号**
```
✗ 123 456 7890
  理由: ハイフンではなくスペースで区切られている
  
✗ 1234567890
  理由: 区切り文字がない
  
✗ (123)456-7890
  理由: カッコの後にスペースがない
  
✗ 12-345-6789
  理由: 最初のグループが2桁(3桁が必要)
```

---

### 5. 実行例

```bash
# file.txtを作成
$ cat > file.txt << EOF
987-123-4567
123 456 7890
(123) 456-7890
EOF

# スクリプトを実行
$ grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt

# 出力
987-123-4567
(123) 456-7890
```

---

## まとめ

**最適解:**
```bash
grep -E '^([0-9]{3}-[0-9]{3}-[0-9]{4}|\([0-9]{3}\) [0-9]{3}-[0-9]{4})$' file.txt
```

この一行で:
- 2つの有効な形式を検出
- 行頭と行末の厳密なマッチング
- シンプルで効率的な実装

が実現できます!